In [7]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from  more_itertools import unique_everseen
import itertools
from datetime import datetime as dt
from urllib.request import *
import re
import operator

In [8]:
pd.options.mode.chained_assignment = None

In [3]:
#get links from reuters

headlinelinks=[]
headlinetitles=[]
timestamps=[]

url = ["https://www.reuters.com/news/archive/mergersNews?view=page&page=1&pageSize=10","https://www.reuters.com/news/archive/EUmergersNews?view=page&page=1&pageSize=10"]

for i in range(20):
    new_page = "page="+str(i)
    for j in url:
        new_url = j
        new_url = new_url.replace("page=1",new_page)
        html = urlopen(new_url)
        soup = bs(html,'html.parser')
        headlines = soup.find_all('div',class_='story-content')
        headlinelinks.extend( [x.find('a')['href'] for x in headlines] )
        headlinetitles.extend( [x.find('h3').text for x in headlines] )
        datetm = [x.find('span',class_='timestamp').text for x in headlines]
        datetm = [re.sub(' EDT','',x) for x in datetm]
        timestamps.extend(datetm)

In [31]:
# get links seekingalpha

headlinelinks=[]
headlinetitles=[]
timestamps=[]

url = ["https://seekingalpha.com/market-news/m-a?current_tag=m-a&until=1508525140"]

# use hdr or hdr2 depending on which one is not giving HTTP 403. If both give then wait 5 minutes and try again
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

hdr2 = {
"Accept-Language": "en-US,en;q=0.5",
"User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:40.0) Gecko/20100101 Firefox/40.0",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
"Referer": "http://thewebsite.com",
"Connection": "keep-alive" 
}

for i in range(1):
    new_page = "page="+str(i)
    for j in url:
        new_url = j
        #new_url = new_url.replace("page=1",new_page)
        req = Request(new_url, headers=hdr)
        html = urlopen(req).read()
        soup = bs(html,'html.parser')
        headlines = soup.find_all('div',class_='title')
        headlinelinks.extend( [x.find('a')['href'] for x in headlines] )
        headlinetitles.extend( [x.find('a').text for x in headlines] )
        #datetm = [x.find('span',class_='timestamp').text for x in headlines]
        #datetm = [re.sub(' EDT','',x) for x in datetm]
        #timestamps.extend(datetm)

In [37]:
timestamps=[]
timelist = soup.find_all('li',class_='date-title')
datetm = [x.text for x in timelist]
datetm = [re.sub(',','',x) for x in datetm]
timestamps.extend(datetm)
timestamps

['Friday October 20 2017',
 'Thursday October 19 2017',
 'Wednesday October 18 2017',
 'Tuesday October 17 2017',
 'Monday October 16 2017',
 'Friday October 13 2017',
 'Thursday October 12 2017',
 'Wednesday October 11 2017',
 'Tuesday October 10 2017']

In [4]:
removelist=['$','million','mln','']
headlinetitles = [x.replace("\t","") for x in headlinetitles]
headlinetitles = [x.replace("\n","") for x in headlinetitles]
#headlinetitles = list(unique_everseen(headlinetitles))
cleanedheadlinetitles = [re.sub("'s","",x) for x in headlinetitles]
cleanedheadlinetitles = [re.sub('[^a-zA-Z|\s]',' ',x) for x in cleanedheadlinetitles]
cleanedheadlinetitles = [re.sub('\s(million|mln|billion|bln)\s',' ',x) for x in cleanedheadlinetitles]
cleanedheadlinetitles = [re.sub('\s(a|to|be)\s',' ',x) for x in cleanedheadlinetitles]
cleanedheadlinetitles = [re.sub('\s+',' ',x) for x in cleanedheadlinetitles]
cleanedheadlinetitles = [re.sub('BRIEF','',x) for x in cleanedheadlinetitles]
cleanedheadlinetitles = [re.sub('UPDATE','',x) for x in cleanedheadlinetitles]
cleanedheadlinetitles = [x.strip() for x in cleanedheadlinetitles]

In [5]:
newtimestamps = []
for item in timestamps:
    try:
        newtimestamps.append(dt.strptime(item,'%b %d %Y'))
    except:
        item = re.sub(r'(am|pm)',r' \1',item)
        currentime = dt.strptime(item,'%I:%M %p')
        currentdate = dt.today()
        currentdate = currentdate.replace(hour = currentime.hour, minute = currentime.minute)
        newtimestamps.append(currentdate)

In [6]:
headlinesframe=[]
headlinesframe = pd.DataFrame(cleanedheadlinetitles,columns=['titles'])
headlinesframe['link'] = pd.DataFrame(headlinelinks)
headlinesframe['datetime'] = pd.DataFrame(newtimestamps)
headlinesframe.sort_values('datetime',ascending=False,inplace=True)
headlinesframededup = headlinesframe.drop_duplicates(subset=['titles'], keep='first')

In [7]:
namelist = pd.read_excel('NameList.xlsx','ModNamesList')
namelistmod = namelist.iloc[:,1:]
searcharray = namelistmod.as_matrix()
newsearcharray=[]
for row in searcharray:
    newrow = [x.strip() for x in row if not pd.isnull(x)]
    newsearcharray.append(newrow)

In [8]:
def checknames(headlines):
    rowname = []
    names = []

    for new in headlines:
        words = new.split()
        s = " "
        newwords=[]    
        for i in range(int(len(words))):
            for j in range(1,5):
                newwords.append(s.join(words[i:i+j])) 
        newwords = list(sorted(set(newwords),key=lambda x:newwords.index(x)))
        newwords2 = sorted(newwords,key = lambda x:(len(x.split()),len(newwords)-newwords.index(x)),reverse=True)
    
        cos = []
        foundwords=[]
    
        for word in newwords2:
        
            founds = [ix for ix, row in enumerate(newsearcharray) for iy, el in enumerate(row) if el.strip().lower() == word.lower()]          
            foundd = [rowd for ix,rowd in enumerate(foundwords) if rowd.strip().lower() == word.lower()]
        
            if len(foundd)!=0:
                continue
        
            if len(founds)!=0:
                cos.extend(founds)
                neww=[]
                w = word.split()
                for i in range(int(len(w))):
                    for j in range(1,5):
                        neww.append(s.join(w[i:i+j])) 
                neww = list(set(neww))
                foundwords.extend(neww)
                continue
        
        cos = list(sorted(set(cos),key=lambda x:cos.index(x)))
        names.append([newsearcharray[i][0] for i in cos])
    
    return(names)

In [9]:
def check_mna(linknames):
    relevant = []
    wordlist = ["buy","bought","buying","acquire","acquired","acquires","acquisition","merge","merger","merged","spinoff","buyout","lbo","stake","sale","divest","divesture"]
    for link in linknames:
        founds = [ix for ix, row in enumerate(wordlist) if row in link.lower()]
        if len(founds)!=0:
            relevant.append(True)
        else:
            relevant.append(False)
    return(relevant)

In [10]:
companeslist = checknames(headlinesframededup['titles'])
relevantlist = check_mna(headlinesframededup['titles'])
headlinesframededup['companies']=companeslist  
headlinesframededup['relevant']=relevantlist
headlinesframededup['refdCount'] = [len(x) for x in headlinesframededup['companies']]

In [11]:
headlinesframerel = headlinesframededup.copy()
headlinesframerel = headlinesframerel[headlinesframerel.relevant==True]
headlinesframerel = headlinesframerel[headlinesframerel.refdCount>0]

In [12]:
headlinesframerel

,titles,link,datetime,companies,relevant,refdCount
22,Civitas Solutions announces acquisition of Men...,/article/brief-civitas-solutions-announces-acq...,2017-10-25 17:35:28.225389,[CIVITAS SOLUTIONS],True,1
25,One Liberty acquires corporate headquarters fo...,/article/brief-one-liberty-acquires-corporate-...,2017-10-25 16:58:28.225489,[ONE LIBERTY PROPERTIES],True,1
26,Endo International completes divestiture of Gr...,/article/brief-endo-international-completes-di...,2017-10-25 16:55:28.225524,[ENDO INTERNATIONAL PLC],True,1
41,Vistra Energy in talks buy Dynegy WSJ,/article/dynergy-inc-ma-vistra-energy/vistra-e...,2017-10-25 16:00:28.226173,"[VISTRA ENERGY CORP, DYNEGY INC]",True,2
49,Activist investor Snow Park urges Cedar explor...,/article/cedar-realty-snow-park/update-1-activ...,2017-10-25 11:47:28.226437,[CEDAR REALTY TRUST INC],True,1
62,The Trade Desk acquires the assets of Adbrain Ltd,/article/brief-the-trade-desk-acquires-the-ass...,2017-10-25 11:01:28.226862,[THE TRADE DESK INC],True,1
63,Sprint quarterly loss narrower than estimated ...,/article/sprint-results/update-3-sprints-quart...,2017-10-25 10:58:28.226894,[SPRINT CORP],True,1
65,T Mobile Sprint ready board committees decide ...,/article/sprint-corp-ma-t-mobile-us/update-1-t...,2017-10-25 10:38:28.226959,[SPRINT CORP],True,1
67,Grubhub will opportunistic around acquisitions...,/article/brief-grubhub-will-be-opportunistic-a...,2017-10-25 10:27:28.227024,[GRUBHUB INC],True,1
81,High Performance Beverages says Cascade Dairy ...,/article/brief-high-performance-beverages-says...,2017-10-25 08:40:28.227407,[PERFORMANCE FOOD GROUP],True,1


In [13]:
### for debug
news = ["Apple to buy Marriott Marriott"]
names=[]

for new in news:
    words = new.split()
    s = " "
    newwords=[]    
    for i in range(int(len(words))):
        for j in range(1,6):
            newwords.append(s.join(words[i:i+j])) 
    newwords = list(sorted(set(newwords),key=lambda x:newwords.index(x)))
    newwords2 = sorted(newwords,key = lambda x:(len(x.split()),len(newwords)-newwords.index(x)),reverse=True)
    
    cos = []
    foundwords=[]
    
    
    for word in newwords2:
        
        founds = [ix for ix, row in enumerate(newsearcharray) for iy, el in enumerate(row) if el.strip().lower() == word.lower()]          
        foundd = [rowd for ix,rowd in enumerate(foundwords) if rowd.strip().lower() == word.lower()]
        
        if len(foundd)!=0:
            continue
        
        if len(founds)!=0:
            cos.extend(founds)
            neww=[]
            w = word.split()
            for i in range(int(len(w))):
                for j in range(1,5):
                    neww.append(s.join(w[i:i+j])) 
            neww = list(set(neww))
            foundwords.extend(neww)
            continue
        
    cos = list(sorted(set(cos),key=lambda x:cos.index(x)))
    names.append([newsearcharray[i][0] for i in cos])
names


[['APPLE INC', 'MARRIOTT INTL INC', 'MARRIOTT VAC WORLD CORP']]

In [12]:
#save output
writer = pd.ExcelWriter('output4.xlsx')
headlinesframededup.to_excel(writer)



    Only loading the 'en' tokenizer.

